## load data 

In [1]:
import scipy.io
import scipy.stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from os import chdir
chdir('/Users/etiennelenaour/Documents/Cours 19:20/Semestre 1/Monte Carlo/projet_HMM/car_park/Cp_16_05_01')

In [3]:
#Telechargement des données pour l'experience 1 (qui sont sous forme de dictionnaire)
exp_ind = 1
gps1 = scipy.io.loadmat('juan{}_gps.mat'.format(exp_ind))
sensors1 = scipy.io.loadmat('juan{}_etc.mat'.format(exp_ind))
laser1 = scipy.io.loadmat('juan{}_lsr2.mat'.format(exp_ind))


#Mise en forme en Data Frame
gps_df = pd.DataFrame(gps1['GPS'], columns=gps1['GPSFieldsOrder'])
sensors_df = pd.DataFrame(sensors1['SENSORS'], columns=sensors1['ETCFieldsOrder'])
laser_df = pd.DataFrame(laser1['LASER'], index=laser1['TLsr'].flatten())
time_laser_vals = laser1['TLsr'].flatten()

In [4]:
#Bien mettre en forme les données 

def readSensorData(sensors_df):
    '''Velocity and steering'''
    # Sensor params
    KV1 = 0.024970 * (1 - 0.21)
    KA1 = 0.00040 * (1 + 0)
    KA0 = 2022
    Kx1 = 1.0127
    Kx2 = 0.0042
    
    time_sensors = sensors_df['time   '].values / 1000.
    steering = Kx1 * (sensors_df['steer  '].values - KA0) * KA1 + Kx2 
    velocity = KV1 * sensors_df['speed  '].values
    
    return time_sensors, steering, velocity


def readGPSData(gps_df):
    '''Latitude and longtitude'''
    # Reference point
    LAT0  = -33.8884
    LONG0 = 151.1948
    
    # Params (wtf is this sorcery)
    a =  6378137.0
    b  = a * (1 - 0.003352810664747)
    kpi = np.pi / 180
    cf = np.cos(LAT0 * kpi)
    sf = np.abs(np.sin(LAT0 * kpi))
    Ro = a ** 2 * cf / np.abs(np.sqrt((a *cf) ** 2 + (b * sf) ** 2))  ;
    RR = b / a  * np.abs(np.sqrt(a ** 2- Ro ** 2)) / sf ;
    
    time_gps = gps_df['1.time   '].values / 1000.
    latitude = - RR * kpi * (gps_df['3.LAT    '].values + LAT0)
    longtitude = Ro * kpi * (gps_df['4.LONG   '].values - LONG0)
    
    return time_gps, latitude, longtitude


def readLaserData(laser_df, time_laser_vals):
    '''
    INUTILE, ON A LE STEERING, VELOCITY ET POSITION(GPS) AVEC CE QUI PRECEDE
    Si j'ai bien compris:
    laser_time: le temps
    laser_range: pour chaque angle, indique la distance a l'objet le plus proche
    laser_intensity: pour chaque angle, l'inteniste lumineuse positive
    '''
    mask13 = 2 ** 13 - 1
    maskA = ~mask13
    length, angles = laser_df.shape
    laser_range = laser_intensity = np.zeros((length, angles))

    for i in range(length):
        laser_obs = laser_df.iloc[i].values
        laser_range[i] = np.array(laser_obs & mask13) # prendre en compte la reflexion des objets (?)
        laser_intensity[i] = np.array(laser_obs & mask13) # apparemment ca sert juste a s'assurer que c'est positif
    
    laser_range = laser_range / 100. # conversion en metres
    time_laser = time_laser_vals / 1000. # ms
    return time_laser, laser_range, laser_intensity

In [5]:
def scanData():
    '''
    Pour avoir le temps, le sensor et l'indice de chaque mesure qui arrive
    Au temps time[i] on a une mesure de sensor[i]
    (1 si gps, 2 si steering et velocity, 3 si laser)
    dans l'indice index[i] du tableau correspondant
    '''
    time_sensors, steering, velocity = readSensorData(sensors_df)
    time_gps, latitude, longtitude = readGPSData(gps_df)
#     time_laser, laser_range, laser_intensity = readLaserData(laser_df, time_laser_vals)
    
    times = [(1, i, time) for i, time in enumerate(time_gps)]
    times.extend([(2, i, time) for i, time in enumerate(time_sensors)])
#     times.extend([(3, i, time) for i, time in enumerate(time_laser)])

    times = np.array(sorted(times, key=lambda u: u[2]))
    time, sensor, index = times[:, 2], times[:, 0], times[:, 1]
    return time, sensor, index

In [6]:
time, sensor, index = scanData()

## Nouveau à partir d'ici

In [7]:
#Construction des bases de données finales 


GPSData = pd.DataFrame(readGPSData(gps_df), ['times_GPS', 'latitude', 'longitude']).T
SensorData = pd.DataFrame(readSensorData(sensors_df), ['time_sensors', 'steering', 'velocity']).T

In [8]:
GPSData.head()

,times_GPS,latitude,longitude
0,851.227,-5.707890,6.615680
1,851.428,-5.709734,6.615680
2,851.628,-5.702355,6.614138
3,851.828,-5.700510,6.615680
4,852.028,-5.702355,6.615680


In [9]:
SensorData.head()

,time_sensors,steering,velocity
0,851.292,0.064557,0.0
1,851.317,0.064962,0.0
2,851.342,0.064557,0.0
3,851.367,0.064557,0.0
4,851.392,0.064557,0.0


In [20]:
vecteur_temps = np.sort(np.array(GPSData['times_GPS'].append(SensorData['time_sensors']).unique()))

In [11]:
def initilisation_pf(N): #on initialise avec des uniformes sur les espaces de vie des variables
    
    init_state = {
        'x': np.random.uniform(low=-20, high=10, size=N),
        'y': np.random.uniform(low=-2.5, high=15, size=N),
        'v': np.random.uniform(low=0, high=4, size=N),
        'psi': np.random.uniform(low=0, high=2*np.pi, size=N),
        'beta': np.random.uniform(low=-0.6, high=0.6, size=N),
        'beta_dot': np.random.uniform(low=-0.6, high=0.6, size=N)
    }
    
    init_poids = [1./N for i in range(N)]
    
    return init_state, init_poids   

In [12]:
# Geometric constants
L = 2.83
H = 0.76
a = 3.78
b = 0.5

def transition_model(current_state, dt):
    '''return the state transition model without taking into
    account he state noise (v' and beta'')'''
    # Retrieve initial states
    x = current_state['x']
    y = current_state['y']
    v = current_state['v']

    psi = current_state['psi']
    beta = current_state['beta']
    beta_dot = current_state['beta_dot']
    
    # Next states
    next_state = dict()
    next_state['x'] = x + dt * v * (np.cos(psi) + (- a * np.sin(psi) + b * np.cos(psi)) * np.tan(beta) / L)
    next_state['y'] = y + dt * v * (np.sin(psi) + (a * np.cos(psi) + b * np.sin(psi)) * np.tan(beta) / L)
    next_state['v'] = v
    
    next_state['psi'] = psi + dt * v * np.tan(beta) / L
    next_state['beta'] = beta + dt * beta_dot
    next_state['beta_dot'] = beta_dot
    
    return next_state   

In [58]:
def update_model(N, time, update_sensor1, update_sensor2, particles, weights, var1, var2):
    
    #Cas GPS
    if update_sensor1:
        
        erreur_mesure = np.linalg.norm([particles['x'] - GPSData.set_index('times_GPS').loc[time].values[0], 
                                        particles['y'] - GPSData.set_index('times_GPS').loc[time].values[0]], axis=0)
        

        for i in range(N): #mise à jour des poids
    
            weights[i] *= scipy.stats.norm(0, var1).pdf(erreur_mesure[i])
        
        weights /= sum(weights) # normalize    
        
        
        
     #Cas sensors   
    if update_sensor2:
        
        erreur_mesure = np.linalg.norm([particles['beta'] - SensorData.set_index('time_sensors').loc[time].values[0], 
                                        particles['v'] - SensorData.set_index('time_sensors').loc[time].values[1]], axis=0)
                                        
        for i in range(N): #mise à jour des poids
    
            weights[i] *= scipy.stats.norm(0, var2).pdf(erreur_mesure[i])
        
        weights /= sum(weights) # normalize  
        
    return weights
        


In [50]:
def simple_resample(particles, weights):
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1. # avoid round-off error
    indexes = np.searchsorted(cumulative_sum, random(N))

    # resample according to indexes
    particles[:] = particles[indexes]
    weights.fill(1.0 / N)
    

In [51]:
def predictions(weights, particles): #Renvoi la prédiction de l'état à chaque temps
    tab_predict = [sum(particles['x'] * weights), sum(particles['y'] * weights), sum(particles['v'] * weights), 
     sum(particles['beta'])]
    
    return tab_predict

In [82]:
#Définissons maintenant le corps de l'algo avec les différentes fonctions crées

list_predict=list()

def particle_filter(N, vecteur_temps=vecteur_temps):
    
    
    #Initialisation de l'algo
    particles, weights = initilisation_pf(N)
    
    #Allons à travers le temps 
    
    for time in vecteur_temps:
        
        particles = transition_model(particles, dt=1/40) #On verra plus tard pour le dt
        
        
        if time in GPSData['times_GPS'].values:#On test si on update pour le GPS
            update_sensor1=True
        
        else:
            update_sensor1=False
            
            
        if time in SensorData['time_sensors'].values:#On test si on update pour les sensors
            update_sensor2=True
        
        else:
            update_sensor2=False
            
        
        weights = update_model(N, time, update_sensor1, update_sensor2, particles, weights, var1=20, var2=20)
        
        list_predict.append(predictions(weights, particles))


In [83]:
#Test update model

particle_filter(N=10, vecteur_temps=vecteur_temps)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
